In [1]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import scipy.optimize as sopt
import pandas as pd
import os
from pyomo.environ import *
from idaes.core.solvers import get_solver
import pandas as pd

In [2]:
# got it means that it is in the excel sheet already

# Q_a = float(Qa[index])   #got it
# Q_b = float(Qb[index])   # got it
# T_e = Te[index]          # got it
# t_a = float(time_col[index,0]) #seconds    got it
# t_b = float(time_col[index,1]) #seconds    got it
# t_c = float(1/freq[index]-t_a-t_b)        #will be calculated in place
# t_b = t_c+t_b                             # will be calcullated in place
# V_p_ta = float(Vipa[index]/t_a) #volts  #will be calculated in place
# V_p_tb = float(Vipb[index]/(t_b+t_c))  #volts   # will be calculated in place
# V_p_tc = 0.0                                   # will be assigned in place
# Flow = flowrate[index]*6*(10**-5)-0.0001 #convert from % to m^3/s, see exccel "flow_calibration..."
# E = float(E_abs[index])                       # got it
# ns = width_ns[index]                          # got it
# temp_C_gas = temp_C[index]                    # will be set.
# frequency = freq[index]                       # got it
# volt = voltage[index]                         #got it
# sign = signature[index]                       # got it

In [3]:
from PyDBDdim.utils import DatasetProcess
DBD = DatasetProcess(r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs_cleanup\data\data_packing\data_23mm_packing_5cm\23mm_packing_5cm")
signature_list = DBD.folder_col
time_col, _= DBD.current_segment()
time_col = np.array(time_col)
t_index = np.array(_)
Vipab, Qab = DBD.get_two_region_VI()
Pabs_kj = DBD.int_power #this is actually worthless but just being complete.
E_abs = DBD.get_plasma_E() #joules?
#print(Vipab)
#print(Qab)
#print(time_col)
#print(_)
#print(signature_list)

Vipa = np.reshape(Vipab[:,0],(Vipab.shape[0],1))
Vipb = np.reshape(Vipab[:,1],(Vipab.shape[0],1))
Qa = np.reshape(Qab[:,0],(Qab.shape[0],1))
Qb = np.reshape(Qab[:,1],(Qab.shape[0],1))
t_a = np.reshape(time_col[:,0],(Qab.shape[0],1))
t_b = np.reshape(time_col[:,1],(Qab.shape[0],1))
Pabs_kj = np.reshape(Pabs_kj, (Pabs_kj.shape[0],1))
E_abs = np.reshape(E_abs, (E_abs.shape[0],1))
print(Pabs_kj.shape)
print(t_a.shape)
print(t_b.shape)
print(Qa.shape)
print(Qb.shape)
print(Vipa.shape)
print(Vipb.shape)
print(E_abs.shape)



matrix = np.hstack((Vipa, Vipb, Qa, Qb, Pabs_kj, E_abs, t_a, t_b))
variables = ['Vipa_Vs', 'Vipb_Vs', 'Qa_As', 'Qb_As', 'Pabs_kJ','E_abs','ta_s', 'tb_s']

(48, 1)
(48, 1)
(48, 1)
(48, 1)
(48, 1)
(48, 1)
(48, 1)
(48, 1)


In [4]:
# check_signal: gets the signal, ,surprise
# check processes signal: 

In [5]:
print(matrix.shape)
print(Qa.shape)
print(Vipa.shape)

(48, 8)
(48, 1)
(48, 1)


In [6]:
print(signature_list)

['OES_20231105194213', 'OES_20231105195119', 'OES_20231105200637', 'OES_20231105201352', 'OES_20231105202216', 'OES_20231105203018', 'OES_20231105203910', 'OES_20231105204924', 'OES_20231105221424', 'OES_20231105222243', 'OES_20231105223103', 'OES_20231105225458', 'OES_20231105230825', 'OES_20231105231639', 'OES_20231105232402', 'OES_20231105233255', 'OES_20231105234050', 'OES_20231105235251', 'OES_20231106000226', 'OES_20231106001835', 'OES_20231106002950', 'OES_20231106004401', 'OES_20231106005231', 'OES_20231106010019', 'OES_20231106010805', 'OES_20231106011718', 'OES_20231106013450', 'OES_20231106014231', 'OES_20231106015741', 'OES_20231106020627', 'OES_20231106021355', 'OES_20231106123710', 'OES_20231106124410', 'OES_20231106125157', 'OES_20231106161350', 'OES_20231106162137', 'OES_20231106162953', 'OES_20231106163750', 'OES_20231106164614', 'OES_20231106180007', 'OES_20231106181053', 'OES_20231106182852', 'OES_20231106183938', 'OES_20231106204029', 'OES_20231106205420', 'OES_2023

In [7]:
# Put this in three parts:
#1) generate data frame from data above
#2) generate data frame from existing excel with data inputs and Temperature
#3) delete redundant, empty columns in old excel
#4) pd.merg the data frames
#5) load into a new excel


#1 df
df0 = pd.DataFrame(data=matrix,columns=[variables[i] for i in range(0,len(variables))]) #first sheet, list dimension vector
df0['signature_dbd'] = signature_list
#df1 = pd.DataFrame(data=signature_list,columns=['signature']) #second sheet, list nullspace vectors and associated variables



# df from existing excel

file_path = r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs_cleanup\data\data_packing\data_23mm_packing_5cm\data_col_vvm.xlsx"
df1 = pd.read_excel(file_path,sheet_name = r"5cm_23mm_good")  #, sheet_name = r"3cm_23mm_good"
df1 = df1.drop(columns = variables, errors = 'ignore')

#merge them
merged_df = pd.merge(df1, df0, left_on="signature", right_on="signature_dbd", how="inner")

merged_df.to_excel(
    r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs_cleanup\data\data_packing\data_23mm_packing_5cm\data_col_5cm_23mm_dbd_recheck.xlsx", 
    sheet_name = r"5cm_23mm",
    index=False)

In [8]:
df1

,width_ns,freq_kHz,voltage_kV,temp_C,height,packing size,power_w,706_667_587_K,706_587_K,706_728_K,706_728_587_K,signature,RF_direct_best,mis
0,180,10,12,na,5,2.3,NaN,NaN,NaN,1384,NaN,OES_20231105194213,NaN,NaN
1,180,7,14,na,5,2.3,NaN,NaN,NaN,1491,NaN,OES_20231105195119,NaN,NaN
2,260,7,10,na,5,2.3,NaN,NaN,NaN,1594,NaN,OES_20231105222243,NaN,NaN
3,260,10,10,na,5,2.3,NaN,NaN,NaN,1566,NaN,OES_20231105223103,NaN,NaN
4,140,5,12,na,5,2.3,NaN,NaN,NaN,1949,NaN,OES_20231105234050,NaN,NaN
5,140,10,12,na,5,2.3,NaN,NaN,NaN,1706,NaN,OES_20231105235251,NaN,NaN
6,180,10,12,na,5,2.3,NaN,NaN,NaN,1703,NaN,OES_20231106004401,NaN,NaN
7,220,3,12,na,5,2.3,NaN,NaN,NaN,2159,NaN,OES_20231106010019,NaN,NaN
8,220,10,12,na,5,2.3,NaN,NaN,NaN,2203,NaN,OES_20231106010805,NaN,NaN
9,100,6,14,na,5,2.3,NaN,NaN,NaN,1940,NaN,OES_20231106013450,NaN,NaN
